# Base Case

In this notebook the basecase which is dissaggregated over locations and time, is made, cleaned and saved.

In [41]:
from ema_workbench import load_results
from ema_workbench import (Model, RealParameter, ScalarOutcome, MultiprocessingEvaluator, 
                           ema_logging, Constant, Scenario, Policy, CategoricalParameter, IntegerParameter,
                           perform_experiments, SequentialEvaluator)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging, utilities
import time

ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [36]:
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation

dike_model,  planning_steps = get_model_for_problem_formulation(5) #disaggregate over locations and time

In [3]:
# we do not want any policies to run for the base case

policies = [Policy("base case no policy", **{k.name: 0 for k in dike_model.levers})] 

In [4]:
# EMA workbench experiment runs with 1000 experiments
n_scenarios = 1000
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios,
                                            policies)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 1000 scenarios * 1 policies * 1 model(s) = 1000 experiments
  0%|                                       | 1/1000 [00:03<1:01:57,  3.72s/it]Traceback (most recent call last):
  File "C:\Users\31649\anaconda3\lib\site-packages\ema_workbench\em_framework\callbacks.py", line 246, in _store_outcomes
    self.results[outcome][case_id,] = outcome_res
KeyError: 'A.1_Dike Investment Costs 0'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\31649\anaconda3\lib\site-packages\ema_workbench\em_framework\ema_multiprocessing.py", line 253, in run
    self.callback(*result.get())
  File "C:\Users\31649\anaconda3\lib\site-packages\ema_workbench\em_framework\callbacks.py", line 281, in __call__
    self._store_outcomes(experiment.experiment_id, outcomes)
  File "C:\Users\31649\anaconda3\lib\site-packages\ema_workbench\em_framework\callbacks.py", line 260, 

In [5]:
#saving the results
utilities.save_results(results, 'results/base_case_diss_time_and_locations_no_policy.tar.gz')

[MainProcess/INFO] results saved successfully to C:\Users\31649\Documents\Master\Model-based decisionmaking\epa1361_open-master\final assignment\results\base_case_diss_time_and_locations_no_policy.tar.gz


In [27]:
import pandas as pd
#Load in the results
base_case_results=utilities.load_results('results/base_case_diss_time_and_locations_no_policy.tar.gz')
experiments, outcomes = base_case_results
outcomes = pd.DataFrame(outcomes)
experiments = pd.DataFrame(experiments)
base_case_results = outcomes.join(experiments)

[MainProcess/INFO] results loaded successfully from C:\Users\31649\Documents\Master\Model-based decisionmaking\epa1361_open-master\final assignment\results\base_case_diss_time_and_locations_no_policy.tar.gz


In [28]:
base_case_results

,A.1_Expected Annual Damage 0,A.1_Dike Investment Costs 0,A.1_Expected Number of Deaths 0,A.2_Expected Annual Damage 0,A.2_Dike Investment Costs 0,A.2_Expected Number of Deaths 0,A.3_Expected Annual Damage 0,A.3_Dike Investment Costs 0,A.3_Expected Number of Deaths 0,A.4_Expected Annual Damage 0,...,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,scenario,policy,model
0,1.607386e+07,0,NaN,NaN,-786209616,NaN,NaN,-786302384,NaN,NaN,...,0,0,0,0,0,0,0,0,base case no policy,dikesnet
1,0.000000e+00,0,0.000000,1.650834e+07,647,NaN,NaN,647,NaN,NaN,...,0,0,0,0,0,0,0,1,base case no policy,dikesnet
2,5.750082e+07,0,0.049468,3.101373e+06,0,0.003643,3.097325e+07,-755124880,NaN,NaN,...,0,0,0,0,0,0,0,2,base case no policy,dikesnet
3,7.562237e+08,0,0.605701,0.000000e+00,0,0.000000,0.000000e+00,0,0.000000,0.000000e+00,...,0,0,0,0,0,0,0,3,base case no policy,dikesnet
4,1.565978e+08,0,0.081475,0.000000e+00,0,0.000000,8.713140e+08,0,1.092142,0.000000e+00,...,0,0,0,0,0,0,0,4,base case no policy,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3.439976e+07,0,0.029896,2.361895e+07,0,0.027094,5.379326e+08,0,1.100780,0.000000e+00,...,0,0,0,0,0,0,0,995,base case no policy,dikesnet
996,4.239234e+07,0,0.029252,1.252469e+07,0,0.011620,2.195979e+07,0,0.038689,1.365650e+07,...,0,0,0,0,0,0,0,996,base case no policy,dikesnet
997,9.373940e+08,0,0.596218,5.252359e+06,0,0.004693,0.000000e+00,0,0.000000,0.000000e+00,...,0,0,0,0,0,0,0,997,base case no policy,dikesnet
998,9.102235e+07,0,0.047832,3.972179e+08,0,0.252305,0.000000e+00,0,0.000000,0.000000e+00,...,0,0,0,0,0,0,0,998,base case no policy,dikesnet


### Cleaning the data

In [29]:
#check whether there are rows that contain NaNs
sum([True for idx,row in base_case_results.iterrows() if any(row.isnull())])

15

In [43]:
#Since 15 of 1000 rows is only 1.5%, we choose to drop these rows

df_correct=base_case_results.dropna().reset_index(drop=True)

df_correct

,A.1_Expected Annual Damage 0,A.1_Dike Investment Costs 0,A.1_Expected Number of Deaths 0,A.2_Expected Annual Damage 0,A.2_Dike Investment Costs 0,A.2_Expected Number of Deaths 0,A.3_Expected Annual Damage 0,A.3_Dike Investment Costs 0,A.3_Expected Number of Deaths 0,A.4_Expected Annual Damage 0,...,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,scenario,policy,model
0,4.036285e+06,0,0.004364,4.554415e+07,0,0.062144,0.000000e+00,0,0.000000,0.000000e+00,...,0,0,0,0,0,0,0,15,base case no policy,dikesnet
1,7.620287e+08,0,0.611532,1.492923e+08,0,0.151080,3.023840e+08,0,0.593600,4.911416e+05,...,0,0,0,0,0,0,0,16,base case no policy,dikesnet
2,9.085729e+08,0,0.581077,0.000000e+00,0,0.000000,0.000000e+00,0,0.000000,0.000000e+00,...,0,0,0,0,0,0,0,17,base case no policy,dikesnet
3,9.286149e+07,0,0.048993,7.073281e+07,0,0.048694,4.188630e+06,0,0.005671,1.175439e+06,...,0,0,0,0,0,0,0,18,base case no policy,dikesnet
4,0.000000e+00,0,0.000000,7.091085e+07,0,0.078947,0.000000e+00,0,0.000000,3.294457e+07,...,0,0,0,0,0,0,0,19,base case no policy,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980,3.439976e+07,0,0.029896,2.361895e+07,0,0.027094,5.379326e+08,0,1.100780,0.000000e+00,...,0,0,0,0,0,0,0,995,base case no policy,dikesnet
981,4.239234e+07,0,0.029252,1.252469e+07,0,0.011620,2.195979e+07,0,0.038689,1.365650e+07,...,0,0,0,0,0,0,0,996,base case no policy,dikesnet
982,9.373940e+08,0,0.596218,5.252359e+06,0,0.004693,0.000000e+00,0,0.000000,0.000000e+00,...,0,0,0,0,0,0,0,997,base case no policy,dikesnet
983,9.102235e+07,0,0.047832,3.972179e+08,0,0.252305,0.000000e+00,0,0.000000,0.000000e+00,...,0,0,0,0,0,0,0,998,base case no policy,dikesnet


## Saving the data

In [44]:
df_correct.to_csv("base_case_without_NaN.csv")